In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import timeit
import torch

from context import rf_pool

In [ ]:
# init rfs and data
img_shape = [20,20]
mu, sigma = rf_pool.utils.lattice.init_uniform_lattice(np.array(img_shape)//2, (4,4), 2, sigma_init=2.)
rfs = rf_pool.utils.lattice.mask_kernel_lattice(mu, sigma, img_shape)

a = torch.as_tensor(np.random.rand(10,64,*img_shape), dtype=torch.float32)
b = rfs.clone();

In [ ]:
# get rf_pool output and timing
t0 = []
a0 = a.numpy()
b0 = b.numpy()
for _ in range(1):
    s = timeit.default_timer()
    out0 = rf_pool.pool.max_pool(a0.reshape((-1,*img_shape)), mask=b0,
                                 kernel=(2,2), img_shape=(20,20))
    e = timeit.default_timer()
    t0.append(e - s)
print('RF-Pool takes %0.4f secs' % np.mean(t0))

In [ ]:
# compare to previous rf_pool approach
t1 = []
for _ in range(1):
    s = timeit.default_timer()
    h_mean = torch.mul(a.unsqueeze(2), b)
    h_sample = torch.reshape(rf_pool.pooling.max_index(h_mean.flatten(-2)), h_mean.shape)
    p_mean = torch.mul(h_mean, h_sample)
    out1 = torch.max(p_mean, -3)[0]
    e = timeit.default_timer()
    t1.append(e - s)
print('Previous approach takes %0.4f secs' % np.mean(t1))

In [ ]:
print('RF-Pool is %0.2f times faster than previous approach' % (np.mean(t1) / np.mean(t0)))

In [ ]:
print('RF-Pool output == previous output: %a' % (np.allclose(out1.numpy(), out0.reshape(out1.shape))))

In [ ]:
# compare to 2x2 maxpool
t2 = []
for _ in range(1000):
    s = timeit.default_timer()
    out2 = torch.nn.functional.max_pool2d(out1, (2,2))
    e = timeit.default_timer()
    t2.append(e - s)
print('2x2 MaxPool takes %0.4f secs' % np.mean(t2))

In [ ]:
print('RF-Pool is %0.2f times faster than 2x2 MaxPool' % (np.mean(t2) / np.mean(t0)))

In [ ]:
print('RF-Pool output == 2x2 MaxPool output: %a' % (np.allclose(out2.numpy(), out0.reshape(out2.shape))))

In [ ]:
def test_fn(a, b):
    return np.mean(a) - np.mean(b)

# bootstrap test difference
diff0 = np.mean(t1) - np.mean(t0)

z_mean = np.mean(t0 + t1)
x_prime = np.array(t0) - np.mean(t0) + z_mean
y_prime = np.array(t1) - np.mean(t1) + z_mean

diffs = rf_pool.utils.functions.bootstrap(x_prime, y_prime, n_samples=1000, fn=test_fn)

print('RF-Pool faster than previous approach: p = %a' % np.mean(diffs > diff0))